
### Creating Your Own Application: `MyImageClassifier`

Creating your own application involves defining a new class that inherits from Deeplay's `Application` class. You can then customize this class to better align with your specific needs. Here's how:

#### Defining Defaults

The `defaults` attribute sets up a `Config` object with default configurations for various components like the backbone, connector, head, loss, and optimizer. These defaults can be overridden when creating a new instance of the class.

In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from deeplay import Application, Config, ConvolutionalEncoder, CategoricalClassificationHead


class MyImageClassifier(Application):

    defaults = (
        Config()
        .backbone(ConvolutionalEncoder)
        .connector(nn.Flatten)
        .head(CategoricalClassificationHead)
        .loss(nn.CrossEntropyLoss)
        .optimizer(torch.optim.Adam, lr=0.001)
    )


**Note**: For convenience, the `defaults` attribute is a class attribute, so you can access it without instantiating the class. This is useful when you want to see what the default configurations are.

### Model attributes

Applications can also have attributes. These are useful for storing information about the model that you might want to access later. For example, you might want to store the number of classes in the dataset, or the size of the input images. You can define these attributes in the default configuration.

In [ ]:
class MyImageClassifier(Application):

    defaults = (
        Config()
        .num_classes(10)
        .backbone(ConvolutionalEncoder)
        .connector(nn.Flatten)
        .head(CategoricalClassificationHead)
        .loss(nn.CrossEntropyLoss)
        .optimizer(torch.optim.Adam, lr=0.001)
    )

In this case, it does not make sense to have a default value for the number of classes, so it will be omitted from the default configuration. Nonetheless, the `head` needs to know the number of classes in the dataset. We want to make sure this is the same as the `num_classes` attribute passed by the user. We can achieve this using the `Ref` object:

In [19]:
from deeplay import Ref

class MyImageClassifier(Application):

    defaults = (
        Config()
        .backbone(ConvolutionalEncoder)
        .connector(nn.Flatten)
        .head(CategoricalClassificationHead, num_classes=Ref('num_classes'))
        .loss(nn.CrossEntropyLoss)
        .optimizer(torch.optim.Adam, lr=0.001)
    )

**Note**: We could also force the user to set `head.num_classes` directly, but doing it this way allows the very user-friendly syntax:

```python	
MyImageClassifier(num_classes=10)
```


#### The Constructor: `__init__`

The constructor function (`__init__`) initializes the various components of the model. In this case, we define the `backbone`, `connector`, and `head` components. We use the `new` method to instantiate these components based on the class or configuration passed during the class initialization.


In [21]:
class MyImageClassifier(Application):

    defaults = (
        Config()
        .backbone(ConvolutionalEncoder)
        .connector(nn.Flatten)
        .head(CategoricalClassificationHead, num_classes=Ref('num_classes'))
        .loss(nn.CrossEntropyLoss)
        .optimizer(torch.optim.Adam, lr=0.001)
    )

    def __init__(self, backbone=None, connector=None, head=None, loss=None, optimizer=None):
        super().__init__(backbone=backbone, connector=connector, head=head, loss=loss, optimizer=optimizer)
        
        # Attributes
        self.num_classes = self.attr("num_classes")

        # Modules
        self.backbone = self.new("backbone")
        self.connector = self.new("connector")
        self.head = self.new("head")

**Note**: We don't need to accept arguments in the constructor function. However, we consider it a good practice to accept arguments of the same name as the components. This helps making the class self-documenting and also allows us allows the alternate model instantiation syntax shown below.

```python
model = MyImageClassifier(backbone=ConvolutionalEncoder, head=CategoricalClassificationHead)
```


#### The Forward Pass: `forward`

Finally, the `forward` method defines how the input data flows through these components. It uses the instantiated objects of the backbone, connector, and head components to process the input and produce the output.



In [24]:
class MyImageClassifier(Application):

    defaults = (
        Config()
        .backbone(ConvolutionalEncoder)
        .connector(nn.Flatten)
        .head(CategoricalClassificationHead, num_classes=Ref('num_classes'))
        .loss(nn.CrossEntropyLoss)
        .optimizer(torch.optim.Adam, lr=0.001)
    )

    def __init__(self, backbone=None, connector=None, head=None, loss=None, optimizer=None):
        super().__init__(backbone=backbone, connector=connector, head=head, loss=loss, optimizer=optimizer)
        
        # Attributes
        self.num_classes = self.attr("num_classes")

        # Modules
        self.backbone = self.new("backbone")
        self.connector = self.new("connector")
        self.head = self.new("head")

    def forward(self, x):
        x = self.backbone(x)
        x = self.connector(x)
        x = self.head(x)
        return x

By following these three main steps—defining defaults, initializing components, and specifying the forward pass—you can create a new, custom application class tailored for your specific needs.

In [25]:
classifier = MyImageClassifier()
print(classifier)

MyImageClassifier(
  (backbone): ConvolutionalEncoder(
    (blocks): ModuleList(
      (0): Template(
        (layer): LazyConv2d(0, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (activation): ReLU()
        (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (1): Template(
        (layer): LazyConv2d(0, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (activation): ReLU()
        (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (2): Template(
        (layer): LazyConv2d(0, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (activation): ReLU()
        (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (3): Template(
        (layer): LazyConv2d(0, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (activation): ReLU()
        (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_

### Extending Application Functionality: Leveraging LightningModule

Deeplay's `Application` class is built upon PyTorch Lightning's `LightningModule`. This means you have the power to override methods like `training_step`, `validation_step`, etc., to add your own custom logic for these stages. This is especially useful for incorporating unique training behaviors or custom metrics.

#### Adding Custom Training and Validation Steps

Below is an example where we extend the `MyImageClassifier` class to include custom training and validation steps:

In [26]:
class MyCustomImageClassifier(MyImageClassifier):

    def training_step(self, batch, batch_idx):
        # Extract data and targets from batch
        x, y = batch

        # Forward pass
        logits = self(x)

        # Compute loss
        loss = self.loss(logits, y)

        # Log training loss
        self.log('train_loss', loss)

        return {'loss': loss}

    def validation_step(self, batch, batch_idx):
        # Extract data and targets from batch
        x, y = batch

        # Forward pass
        logits = self(x)

        # Compute loss
        loss = self.loss(logits, y)

        # Compute accuracy
        preds = torch.argmax(logits, dim=1)
        acc = (preds == y).float().mean()

        # Log validation loss and accuracy
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', acc, prog_bar=True)

        return {'val_loss': loss, 'val_acc': acc}


In this example, we've overridden the `training_step` and `validation_step` methods to include the logging of loss and accuracy metrics. The `self.log` method makes it easy to track these metrics during training and validation. Since `Application` is an extension of `LightningModule`, all the features and methods available in `LightningModule` are naturally available in your custom `Application` class as well.

In [29]:
class MySubclassOfImageClassifier:
    defaults = (
        Config()
        .merge(None, MyImageClassifier.defaults)
        .my_custom_parameter(42)
    )

### Extending Class Defaults with Config

When building your own custom applications, you may want to extend the configurations of a parent class to include additional or modified settings. Deeplay's `Config` object provides a convenient `.merge()` method to facilitate this.

#### Merging at the Root Level

If you want to merge the parent class's defaults into your own at the root level, you can specify `None` as the first argument to `.merge()` like so:


In [30]:
class MySubclassOfImageClassifier:

    defaults = (
        Config()
        .merge(None, MyImageClassifier.defaults)
        .my_custom_parameter(42)
    )


Here, the `None` indicates that `MyImageClassifier.defaults` should be merged into `MySubclassOfImageClassifier`'s `defaults` at the root level.

#### Merging Into a Specific Key

Alternatively, if you want to merge the defaults under a specific key in your configuration, you can specify that key:


In [31]:
defaults = (
    Config()
    .merge("parent_defaults", MyImageClassifier.defaults)
    .my_custom_parameter(42)
)


In this example, `MyImageClassifier.defaults` will be merged under the key `parent_defaults` in the `defaults` configuration for `MySubclassOfImageClassifier`.

This allows you to extend and customize configurations flexibly, while keeping the settings from the parent class accessible and organized.